Начало работы: импорт библиотек, открытие скачанного Excel-файла. В большинстве столбцов информации недостаточно (большое количество NaN-значений), удалим их за ненадобностью.

In [2]:
import numpy as np
import pandas as pd

pd.options.mode.copy_on_write = True

dataset = pd.read_excel('/content/drive/MyDrive/data/1. Excel.xlsx', 1)

dataset.drop(['sprint_name', 'sprint_date_start', 'sprint_date_finish', 'dev_team',	'dev_team_lead',	'product_team', 'date_export', 'task_type'], axis=1, inplace=True)
dataset.rename(columns={'date_update': 'date_last'}, inplace=True)

In [3]:
dataset['id'].count()

13403

Количество записей 13403, для каждого заказа оставим последнюю запись для формирования окончательного статуса - при значении is_last равном 1, оставляем запись, все остальные записи удаляются. После этого, удаляется сам столбец is_last, а также необходимые столбцы отмечаются, как содержащие тип данных datetime.

In [4]:
dataset.astype({'date_creation': 'datetime64[ns]',
                'deadline': 'datetime64[ns]',
                'stage_deadline': 'datetime64[ns]',
                'stage_date': 'datetime64[ns]'
                })

dataset = dataset[dataset['is_last'] == 1].drop('is_last', axis=1)

In [5]:
dataset.head(2)

,id,date_creation,date_last,task_name,task_status,stage,executor_team,priority_general,deadline,microservice,admin_team,techsup_team,stage_deadline,stage_date,marks_str,is_subtask,parent_id,general_type
1,ВВ-36446,2024-01-02 07:14:48,2024-01-15 02:37:50,02.01.24_КМ►При выборе конкретного слота появл...,Отменено,Отменить задание,Группа ВкусВилл Интегратор. TEAM 1,3. Средний,2024-02-16 07:14:48,Корзина,NaN,Группа ВкусВилл Интегратор. TEAM 1,NaT,2024-01-15 02:37:49,Средний приоритет;ТП2: Ждём/собираем примеры,0,NaN,Задача
7,ВВ-36447,2024-01-02 07:19:47,2024-01-26 16:03:13,02.01.24_КМ► При попытке положить товар в корз...,Отменено,Отменить задание,Группа ВкусВилл Интегратор. TEAM 3,4. Низкий,2024-04-01 07:19:47,Каталог,Группа ВкусВилл Интегратор. TEAM 3,Группа ВкусВилл Интегратор. TEAM 3,NaT,2024-01-26 16:03:12,Низкий приоритет;ТП2: Ждём/собираем примеры,0,NaN,Задача


Определим два новых столбца: в одном из них будет храниться информация о том, завершен ли таск вовремя (не просрочены ли дедлайны), во втором количество дней, затраченных на исправление бага.

In [6]:
dataset['completed_on_time'] = 0
dataset['count_days'] = ''

dataset['completed_on_time'] = dataset.apply(lambda x: 1 if (x.date_last <= x.deadline and x.task_status != "В работе") else 0, axis=1)

dataset['count_days'] = dataset.apply(lambda x: x.date_last - x.date_creation if x.task_status == 'Закрыто' else np.nan, axis=1)

In [7]:
dataset[dataset['completed_on_time'] == 1].count().id / dataset.count().id * 100

75.92857142857142

Три четверти задач (75%) выполнены вовремя.

In [8]:
dataset[dataset['count_days'] == dataset['count_days'].max()]

,id,date_creation,date_last,task_name,task_status,stage,executor_team,priority_general,deadline,microservice,admin_team,techsup_team,stage_deadline,stage_date,marks_str,is_subtask,parent_id,general_type,completed_on_time,count_days
498,ВВ-37728,2024-01-11 21:02:06,2024-06-21 11:01:28,11.01.2024LK_► Товары из заказа на завтра не п...,Закрыто,Закрыть задание,NaN,3. Средний,2024-02-25 21:02:06,Распределение,Группа ВкусВилл Интегратор. TEAM 4,Группа ВкусВилл Интегратор. TEAM 4,NaT,2024-06-21 11:01:20,NaN,0,NaN,Задача,0,161 days 13:59:22


Баг, на устранение которого потребовалось больше всего времени.

Сформируем датасеты, содержащие количество найденных и исправленных багов в определенном месяце и экспортируем в Excel-файл.

In [9]:
open_task_for_month = dataset.groupby(dataset.date_creation.dt.month).count()['id']
done_task_for_month = dataset.groupby(dataset.date_last.dt.month).count()['id']

result_for_month = pd.concat([open_task_for_month, done_task_for_month], axis=1, keys=['date_start', 'date_end']).fillna(0)

open_tasks = dataset[['priority_general', 'microservice', 'task_status']]

with pd.ExcelWriter('/content/drive/MyDrive/data/result_g.xlsx') as writer:
  result_for_month.to_excel(writer, 'Количество багов по месяцам', na_rep='-')
  open_tasks.to_excel(writer, 'Все баги', na_rep='-')
  open_tasks.query('task_status == "В работе"').to_excel(writer, 'Незакрытые баги', na_rep='-', columns=['priority_general', 'microservice'])